<a href="https://colab.research.google.com/github/KimJisanER/code_jjambbong/blob/main/add_equevalent_0325.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install rdkit
from rdkit import Chem
from rdkit.Chem import PandasTools
from rdkit.Chem import AllChem
import re

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 27.8 MB/s eta 0:00:00


In [2]:
def load_sdf_file(file_path):
    sanitized_mols = []
    suppl = Chem.SDMolSupplier(file_path)
    for x in suppl:
        if x is not None:
            try:
                x_with_h = Chem.AddHs(x, addCoords=True)
                Chem.SanitizeMol(x_with_h)
                sanitized_mols.append(x_with_h)
            except:
                sanitized_mols.append('')

    df = PandasTools.LoadSDF(file_path)
    df['Molecules'] = sanitized_mols
    return df

In [3]:
# df =load_sdf_file("/content/drive/MyDrive/EBoMD_train_240322_v3.sdf").reset_index(drop=True)
df =load_sdf_file("/content/drive/MyDrive/EBoMD_test_240322_v2.sdf").reset_index(drop=True)

In [4]:
def check_molecule_properties(mol):
    """
    Check properties of a molecule.

    Parameters:
    mol (rdkit.Chem.rdchem.Mol): RDKit molecule object.

    Returns:
    dict: Dictionary containing molecule properties.
    """
    properties = {}
    prop_names = mol.GetPropNames()
    for prop_name in prop_names:
        prop_value = mol.GetProp(prop_name)
        properties[prop_name] = prop_value
    return properties


def extract_characters(input_string):
  # extract map numbers
    pattern = pattern = r'<(\w+),(\w+);'
    matches = re.search(pattern, input_string)
    if matches:
        char_at_1st_position = matches.group(1)
        char_at_2nd_position = matches.group(2)
        return [char_at_1st_position, char_at_2nd_position]
    else:
        return None

def extract_reactions(input_string):
  # extract reaction types
    input_string = input_string.replace(" " , "")
    pattern = r';([\w-]+);([\w\d]+)\>'
    matches = re.search(pattern, input_string)
    if matches:
        reaction_type = matches.group(1)
        reaction_index = matches.group(2)
        return reaction_type, reaction_index
    else:
        return None, None

def get_bonds_with_map_numbers(mol):
    """
    Get all bonds in the molecule with map numbers.

    Parameters:
    mol (rdkit.Chem.rdchem.Mol): RDKit molecule object.

    Returns:
    list: List of sets containing map numbers of atoms forming bonds.
    """
    for idx, atom in enumerate(mol.GetAtoms()):
        atom.SetAtomMapNum(idx + 1)
    bond_list = []
    for bond in mol.GetBonds():
        begin_atom_idx = str(bond.GetBeginAtom().GetAtomMapNum())
        end_atom_idx = str(bond.GetEndAtom().GetAtomMapNum())
        bond_list.append({begin_atom_idx, end_atom_idx})
    return bond_list

def equivalent_map(pair, equivalent_dict):
    rank_bond_pair = []
    for atom_number in pair:
        if atom_number.isalpha():
            rank_bond_pair.append(atom_number)
        else:
            CanonicalRank = equivalent_dict[atom_number]
            rank_bond_pair.append(CanonicalRank)
    return list(rank_bond_pair)

def has_alphabets(input_list):
    return any(isinstance(item, str) and item.isalpha() for item in input_list)

In [5]:
def process_equivalent_label(input_label, mol):
    """
    Process the input label to generate output labels.

    Parameters:
    input_label (str): Input label string.
    mol (rdkit.Chem.rdchem.Mol): RDKit molecule object.

    Returns:
    str: Output label.
    """
    equivalent_dict = {}
    ranked_1 = list(Chem.CanonicalRankAtoms(mol, breakTies=False))
    for i, j in enumerate(ranked_1):
        equivalent_dict[str(i+1)] = str(j+1)

    input_pair = extract_characters(input_label)
    reaction_type, reaction_index = extract_reactions(input_label)
    rank_bond_pair = equivalent_map(input_pair, equivalent_dict)

    equivalent_label = []
    if has_alphabets(rank_bond_pair):
        for key, value in equivalent_dict.items():
            rank_bond_pair_copy = rank_bond_pair.copy()
            if value in rank_bond_pair_copy:
                rank_bond_pair_copy.remove(value)
                rank_bond_pair_copy.append(key)
                equivalent_label.append(sorted(rank_bond_pair_copy))
    else:
        for pair in get_bonds_with_map_numbers(mol):
            if set(rank_bond_pair) == set(equivalent_map(pair, equivalent_dict)):
                equivalent_label.append(sorted(pair))

    out_put_label = ''
    for pair in equivalent_label:
        mapnumber_1 = pair[0]
        mapnumber_2 = pair[1]
        label = f'<{mapnumber_1},{mapnumber_2};{reaction_type};{reaction_index}>\n'
        out_put_label += label

    return out_put_label



In [6]:
BOM_types = ['BOM_1A2', 'BOM_2A6', 'BOM_2B6', 'BOM_2C8', 'BOM_2C9', 'BOM_2C19', 'BOM_2D6', 'BOM_2E1', 'BOM_3A4']

def add_equivalent_label(df, process_equivalent_label):
    import copy

    BOM_types = ['BOM_1A2', 'BOM_2A6', 'BOM_2B6', 'BOM_2C8', 'BOM_2C9', 'BOM_2C19', 'BOM_2D6', 'BOM_2E1', 'BOM_3A4']
    # BOM_types = ['BOM_2E1']

    new_df = copy.deepcopy(df)  # 원본 데이터프레임을 수정하지 않기 위해 deepcopy 사용

    for bom_type in BOM_types:
        print(bom_type)
        for i in range(len(new_df)):
          mol = new_df['Molecules'][i]
          input_labels = [label for label in new_df[bom_type][i].split('\n')]
          output_labels = ''
          print(i)
          for j in range(len(input_labels)):
            input_label = input_labels[j]
            if input_label != '':
              # try:
                output_label = process_equivalent_label(input_label, mol)
                output_labels += output_label
          new_df[bom_type][i] = output_labels
              # except Exception as e:
              #   print('ERROR',e)
          print(output_labels)

    return new_df


In [7]:
new_df = add_equivalent_label(df, process_equivalent_label)

BOM_1A2
0
<1,2;Cleavage;R1>
<1,H;Oxidation;R1>

1
<5,6;Cleavage;R1>
<4,5;Oxidation;R1>

2
<1,2;Cleavage;R1>
<1,H;Oxidation;R1>

3

4
<5,8;Cleavage;R1>
<8,H;Oxidation;R1>
<5,6;Cleavage;R2>
<6,H;Oxidation;R2>

5
<19,20;Cleavage;R1>
<20,H;Oxidation;R1>

6
<14,15;Cleavage;R1>

7

8
<12,S;S-Oxidation;R1>

9
<20,21;Cleavage;R1>
<21,H;Oxidation;R1>

10

11
<5,6;Cleavage;R1>
<6,H;Oxidation;R1>
<10,H;Hydroxylation;R2>
<12,H;Hydroxylation;R3>

12
<11,12;Cleavage;R1>
<12,H;Oxidation;R1>

13
<4,S;S-Oxidation;R1>

14
<2,7;Oxidation;R1>
<2,3;Reduction;R1>
<3,4;Oxidation;R1>
<4,5;Oxidation;R1>
<5,6;Reduction;R1>
<6,7;Oxidation;R1>
<4,H;Oxidation;R1>
<7,H;Oxidation;R1>
<13,H;Hydroxylation;R2>

15

16
<3,7;Cleavage;R1>
<7,H;Oxidation;R1>
<10,4;Cleavage;R2>
<10,H;Oxidation;R2>

17

18

19

20

21

22

23

24

25

26

27

28

29

30

31

32

33

34

35

36

37

38

39

40

41

42

43

44

45

46

47

48

49

50

51

52

53

54

55

56

57

58

59

60

61

62

63

64

65

66

67

68

69

70

71

72

73

7

In [9]:
def save_sdf_file(df, file_path):
    writer = Chem.SDWriter(file_path)
    for i, row in df.iterrows():
        if row['Molecules'] and isinstance(row['Molecules'], Chem.Mol):
            mol = row['Molecules']
            # Update the molecule object with the corresponding properties from the DataFrame
            for prop_name, prop_value in row.items():
                if prop_name != 'Molecules':
                    mol.SetProp(str(prop_name), str(prop_value))
            writer.write(mol)
    writer.close()

save_sdf_file(new_df, '/content/drive/MyDrive/additional_BOM_test.sdf')

In [10]:
new_df

,InChIKey,BOM_1A2,BOM_2A6,BOM_2B6,BOM_2C8,BOM_2C9,BOM_2C19,BOM_2D6,BOM_2E1,BOM_3A4,References,ID,Comment,PubChem_CID,FLAG_COMMENTS,ROMol,Molecules
0,AAKJLRGGTJKAMG-UHFFFAOYSA-N,"<1,2;Cleavage;R1>\n<1,H;Oxidation;R1>\n",,,,,,,,"<1,2;Cleavage;R1>\n<1,H;Oxidation;R1>\n<11,12;...","(R1) Wang, B. and Zhou, S.F. (2009); Synthetic...",Erlotinib,,,,<rdkit.Chem.rdchem.Mol object at 0x7a949595ece0>,<rdkit.Chem.rdchem.Mol object at 0x7a94a231adc0>
1,VHOGYURTWQBHIL-UHFFFAOYSA-N,"<5,6;Cleavage;R1>\n<4,5;Oxidation;R1>\n",,,,"<5,6;Cleavage;R1>\n<4,5;Oxidation;R1>\n",,,,"<5,6;Cleavage;R1>\n<4,5;Oxidation;R1>\n",PMID: 19754423,leflunomide,,,,<rdkit.Chem.rdchem.Mol object at 0x7a949595ed50>,<rdkit.Chem.rdchem.Mol object at 0x7a94a231ad50>
2,QUNWUDVFRNGTCO-UHFFFAOYSA-N,"<1,2;Cleavage;R1>\n<1,H;Oxidation;R1>\n",,,,,,,,,PMID: 8162667;PMID: 19754423,paraxanthine,,,,<rdkit.Chem.rdchem.Mol object at 0x7a949595edc0>,<rdkit.Chem.rdchem.Mol object at 0x7a94a231ae30>
3,CJJOSEISRRTUQB-UHFFFAOYSA-N,,,,,,,,,,"PMID: 21782601;PMID: 12898643 \nBuratti, Franc...",Azinphos-methyl,,,,<rdkit.Chem.rdchem.Mol object at 0x7a949595ee30>,<rdkit.Chem.rdchem.Mol object at 0x7a94a231aea0>
4,PMGUCIDDSCRAMY-UHFFFAOYSA-N,"<5,8;Cleavage;R1>\n<8,H;Oxidation;R1>\n<5,6;Cl...",,,,,,"<5,8;Cleavage;R1>\n<8,H;Oxidation;R1>\n<5,6;Cl...",,"<5,8;Cleavage;R1>\n<8,H;Oxidation;R1>\n<5,6;Cl...","(R3) Zhou, S-F.; (2016); Cytochrome P450 2D6: ...",3-Hydroxylidocaine,,,,<rdkit.Chem.rdchem.Mol object at 0x7a949595eea0>,<rdkit.Chem.rdchem.Mol object at 0x7a94a231af80>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,,,,,,,,,,,,,,,,<rdkit.Chem.rdchem.Mol object at 0x7a949598d850>,<rdkit.Chem.rdchem.Mol object at 0x7a949595ea40>
93,,,,,,,,,,,,,,,,<rdkit.Chem.rdchem.Mol object at 0x7a949598d8c0>,<rdkit.Chem.rdchem.Mol object at 0x7a949595eab0>
94,,,,,,,,,,,,,,,,<rdkit.Chem.rdchem.Mol object at 0x7a949598d930>,<rdkit.Chem.rdchem.Mol object at 0x7a949595eb20>
95,,,,,,,,,,,,,,,,<rdkit.Chem.rdchem.Mol object at 0x7a949598d9a0>,<rdkit.Chem.rdchem.Mol object at 0x7a949595eb90>
